#### Copyright 2020 Google LLC.

# Reformer: Text Generation [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/trax/blob/master/trax/models/reformer/text_generation.ipynb)

This notebook was designed to run on TPU.

To use TPUs in Colab, click "Runtime" on the main menu bar and select Change runtime type. Set "TPU" as the hardware accelerator.

In [ ]:
# Install JAX.
!pip install --upgrade jax
!pip install --upgrade jaxlib
!pip install --upgrade trax

# Make sure the Colab Runtime is set to Accelerator: TPU.
import requests
import os
if 'TPU_DRIVER_MODE' not in globals():
  url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
  resp = requests.post(url)
  TPU_DRIVER_MODE = 1

# The following is required to use TPU Driver as JAX's backend.
from jax.config import config
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
print(config.FLAGS.jax_backend_target)

In [ ]:
!pip install --upgrade -q sentencepiece
!pip install --upgrade -q gin 
!pip install bert_score

from tensorflow.compat.v1.io.gfile import GFile
import gin
import os
import jax
import trax
from trax.data import inputs
import pickle

import numpy as np
import jax.numpy as jnp
from bert_score import score

from scipy.special import softmax

from sentencepiece import SentencePieceProcessor

In [ ]:
# ユーザー関数
def write_text(text, id):
    with open("sample.txt", "a", encoding="utf-8") as f:
        f.write("start_id: {}, text: {}".format(str(id), text))
        f.write("\n")
        
def calc_bert_score(cands, refs):
    """ BERTスコアの算出

    Args:
        cands ([List[str]]): [比較元の文]
        refs ([List[str]]): [比較対象の文]

    Returns:
        [(List[float], List[float], List[float])]: [(Precision, Recall, F1スコア)]
    """
    Precision, Recall, F1 = score(cands, refs, lang="ja", verbose=True)
    return Precision.numpy().tolist(), Recall.numpy().tolist(), F1.numpy().tolist()

def loadPickle(fileName):
    with open(fileName, mode="rb") as f:
        return pickle.load(f)

def dumpPickle(fileName, obj):
    with open(fileName, mode="wb") as f:
        pickle.dump(obj, f)

## Setting up data and model

In this notebook, we'll be pushing the limits of just how many tokens we can fit on a single TPU device. The TPUs available in Colab have 8GB of memory per core, and 8 cores. We will set up a Reformer model that can fit a copy of "Crime and Punishment" on *each* of the 8 TPU cores (over 500,000 tokens per 8GB of memory).

In [ ]:
# corpusデータを読み込み
corpus = loadPickle("corpus.pickle")
# 長文順にソート
# corpus.sort(key=len, reverse=True)
print(len(corpus))

text = "".join(corpus[5000:10000])
print(len(text))

In [ ]:
# Load a BPE vocabulaary with 320 types. This mostly consists of single letters
# and pairs of letters, but it has some common words and word pieces, too.
# !gsutil cp gs://trax-ml/reformer/cp.320.* .

TOKENIZER = SentencePieceProcessor()
TOKENIZER.load('./utils/shogi.model')

In [ ]:
# Tokenize
IDS = TOKENIZER.EncodeAsIds(text)
IDS = np.asarray(IDS, dtype=np.int32)
PAD_AMOUNT = 256 * 512 - len(IDS)
print("Number of tokens:", IDS.shape[0])
print("Number of PAD AMOUNT:", PAD_AMOUNT)

As we see above, "Crime and Punishment" has just over half a million tokens with the BPE vocabulary we have selected.

Normally we would have a dataset with many examples, but for this demonstration we fit a language model on the single novel only. We don't want the model to just memorize the dataset by encoding the words in its position embeddings, so at each training iteration we will randomly select how much padding to put before the text vs. after it.

We have 8 TPU cores, so we will separately randomize the amount of padding for each core.

In [ ]:
# Set up the data pipeline.
def my_inputs(n_devices):
  while True:
    inputs = []
    mask = []
    pad_amounts = np.random.choice(PAD_AMOUNT, n_devices)
    for i in range(n_devices):
      inputs.append(np.pad(IDS, (pad_amounts[i], PAD_AMOUNT - pad_amounts[i]),
                            mode='constant'))
      mask.append(np.pad(np.ones_like(IDS, dtype=np.float32),
                          (pad_amounts[i], PAD_AMOUNT - pad_amounts[i]),
                          mode='constant'))
    inputs = np.stack(inputs)
    mask = np.stack(mask)
    yield (inputs, inputs, mask)

print("(device count, tokens per device) = ",
      next(my_inputs(trax.fastmath.device_count()))[0].shape)

In [ ]:
# Configure hyperparameters.
gin.parse_config("""
import trax.layers
import trax.models
import trax.optimizers
import trax.data.inputs
import trax.supervised.trainer_lib

# Parameters that will vary between experiments:
# ==============================================================================
train.model = @trax.models.ReformerLM
# Our model will have 6 layers, alternating between the LSH attention proposed
# in the Reformer paper and local attention within a certain context window.
n_layers = 6
attn_type = [
  @trax.layers.SelfAttention,
  @LSHSelfAttention,  
  @trax.layers.SelfAttention,
  @LSHSelfAttention,
  @trax.layers.SelfAttention,
  @LSHSelfAttention,
  ]
share_qk = False  # LSH attention ignores this flag and always shares q & k
n_heads = 2
attn_kv = 128
dropout = 0.10
n_tokens = 131072

# Parameters for multifactor:
# ==============================================================================
multifactor.constant = 0.01
multifactor.factors = 'constant * linear_warmup * cosine_decay'
multifactor.warmup_steps = 100
multifactor.steps_per_cycle = 900

# Parameters for Adam:
# Parameters for RMSProp:
# ==============================================================================
Adam.weight_decay_rate=0.0
Adam.b1 = 0.86
Adam.b2 = 0.92
Adam.eps = 1e-9

# Parameters for RMSProp: 試行してみたものの、数100epochでlossが下がらず断念。
# ==============================================================================
# learning_rate=0.001
# gamma=0.9
# eps=1e-08
# clip_grad_norm=None

# Parameters for SelfAttention:
# ==============================================================================
trax.layers.SelfAttention.attention_dropout = 0.10
trax.layers.SelfAttention.chunk_len = 64
trax.layers.SelfAttention.n_chunks_before = 1
trax.layers.SelfAttention.n_parallel_heads = 1

# Parameters for LSHSelfAttention:
# ==============================================================================
LSHSelfAttention.attention_dropout = 0.2
LSHSelfAttention.chunk_len = 64
LSHSelfAttention.n_buckets = [64, 128]
LSHSelfAttention.n_chunks_after = 0
LSHSelfAttention.n_chunks_before = 1
LSHSelfAttention.n_hashes = 1
LSHSelfAttention.n_parallel_heads = 1
LSHSelfAttention.predict_drop_len = 128
LSHSelfAttention.predict_mem_len = 1024

# Parameters for ReformerLM:
# ==============================================================================
ReformerLM.attention_type = %attn_type
ReformerLM.d_attention_key = %attn_kv
ReformerLM.d_attention_value = %attn_kv
ReformerLM.d_model = 256
ReformerLM.d_ff = 512
ReformerLM.dropout = %dropout
ReformerLM.ff_activation = @trax.layers.Relu
ReformerLM.max_len = %n_tokens
ReformerLM.mode = 'train'
ReformerLM.n_heads = %n_heads
ReformerLM.n_layers = %n_layers
ReformerLM.vocab_size = 6000
ReformerLM.axial_pos_shape = (256, 512)
ReformerLM.d_axial_pos_embs= (64, 192)
""")

## 学習時のみ使用

In [ ]:
# Set up a Trainer.
output_dir = os.path.expanduser('~/train_dir/')
# output_dir = os.getcwd()
!rm -f ~/train_dir/model.pkl.gz  # Remove old model

trainer = trax.supervised.Trainer(
    model=trax.models.ReformerLM,
    loss_fn=trax.layers.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam,
    lr_schedule=trax.lr.multifactor(),
    inputs=trax.data.inputs.Inputs(my_inputs),
    output_dir=output_dir)

In [ ]:
# Run one training step, to make sure the model fits in memory.
# The first time trainer.train_epoch is called, it will JIT the entire network
# architecture, which takes around 2 minutes. The JIT-compiled model is saved
# so subsequent runs will be much faster than the first.
trainer.train_epoch(n_steps=1, n_eval_steps=1)

In [ ]:
# Train for 600 steps total
# The first ~20 steps are slow to run, but after that it reaches steady-state
# speed. This will take at least 30 minutes to run to completion, but can safely
# be interrupted by selecting "Runtime > Interrupt Execution" from the menu.
# The language model won't be exceptionally good when trained for just a few
# steps and with minimal regularization. However, we can still sample from it to
# see what it learns.
trainer.train_epoch(n_steps=9, n_eval_steps=1)
for _ in range(300):
  trainer.train_epoch(n_steps=10, n_eval_steps=1)

In [ ]:
cp /root/train_dir/model.pkl.gz ./model.pkl.gz

## 文章生成時に使用

In [ ]:
# As we report in the Reformer paper, increasing the number of hashing rounds
# helps with quality. We can even increase the number of hashing rounds at
# evaluation time only.

gin.parse_config("""LSHSelfAttention.n_hashes = 4""")

In [ ]:
# Load the trained Reformer in 'predict' mode
model = trax.models.ReformerLM(mode='predict')
# model.init_from_file(os.path.join(os.getcwd(),'model.pkl.gz'),
#                      weights_only=True)

model.init_from_file(os.path.join(os.getcwd(),'model/model.pkl.gz'),
                     weights_only=True)
# Sample from ReformerLM
output_token_ids = trax.supervised.decoding.autoregressive_sample(
    model, temperature=0.0)

# Decode token IDs
# Reformer outputed a batch with one item, we access it using [0]
# tolist() converts from int64 to int, the type SentencePiece expects
TOKENIZER = SentencePieceProcessor()
TOKENIZER.load('./utils/shogi.model')

TOKENIZER.DecodeIds(output_token_ids[0].tolist()) 


In [ ]:
# 文章生成
sample_corpus = []
ids = np.random.randint(4, 6000, 300)

for i, id in enumerate(ids):
    print("-------------------processing{}----------------------".format(str(i)))
    # 引数のstart_idかtemperatureを変更することでランダム生成できる
    out = trax.supervised.decoding.autoregressive_sample(model, temperature=0.0, start_id=id, max_length=30)
    sample_text = TOKENIZER.DecodeIds(out[0].tolist())
    sample_corpus.append(sample_text)

## 生成文書を保存したい場合
## pickle形式
# DumpPickle(sample_corpus, "sample_corpus")